In [3]:
import torch
import sys

### Training model

#### Implementing custom dataset

In [4]:
from dataset import LMDBDataset 

sys.path.append('./config')
from train import train_start_index, train_end_index, data_path, batch_size, val_start_index, val_end_index, is_import_model, model_path, num_epochs, save_path

In [5]:
def collate_fn(batch):
    return tuple(zip(*batch))


train_dataset = LMDBDataset(data_path['train'],train_start_index,train_end_index)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True,collate_fn=collate_fn, num_workers=0)

val_dataset = LMDBDataset(data_path['val'],val_start_index,val_end_index)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=True,collate_fn=collate_fn, num_workers=0)

#### Setting Model

In [6]:
from model import FasterRCNNResNet50 

In [ ]:
model =  torch.load(model_path) if is_import_model else FasterRCNNResNet50(num_classes=14)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [8]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

#### Looping train

In [ ]:
for epoch in range(num_epochs):
    model.train()     
       
    running_loss = 0.0
    for i, (images, targets) in enumerate(train_dataloader):
        
        print(i)

        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        optimizer.zero_grad()

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
    
    avg_train_loss = running_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {avg_train_loss}")        

    lr_scheduler.step()


    val_running_loss = 0.0
    with torch.no_grad():
        for i, (images, targets) in enumerate(val_dataloader):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            val_running_loss += losses.item()


    avg_val_loss = val_running_loss / len(val_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Validation Loss: {avg_val_loss}")
    
print("Training complete!")


#### Saving trained model

In [9]:
torch.save(model.state_dict(), save_path)